In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras
import glob
import PIL, cv2
import os
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../scripts/')
sys.path.append('../scripts/helper_functions_cv/tensorflow_helpers/')
from save_weights_every_epoch import CallbackForSavingModelWeights
from swin_transformer import final_model
%load_ext autoreload
%autoreload 2
import tensorflow.keras.backend as K
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

In [2]:
allowed_gpus = [0]
gpus = tf.config.list_physical_devices('GPU')
final_gpu_list = [gpus[x] for x in allowed_gpus]
tf.config.set_visible_devices(final_gpu_list, 'GPU')
for gpu in final_gpu_list:
    tf.config.experimental.set_memory_growth(gpu, True)
strategy = tf.distribute.MirroredStrategy()
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [3]:
csv_file = pd.read_csv('../../files/train.csv')

In [4]:
csv_file.head()

,image_id,center_id,patient_id,image_num,label
0,006388_0,11,006388,0,CE
1,008e5c_0,11,008e5c,0,CE
2,00c058_0,11,00c058,0,LAA
3,01adc5_0,11,01adc5,0,LAA
4,026c97_0,4,026c97,0,CE


In [5]:
csv_file['fixed_paths'] = csv_file['image_id'].apply(lambda x: '../../files/resized_train/' + x + '.png')

In [6]:
csv_file.head()

,image_id,center_id,patient_id,image_num,label,fixed_paths
0,006388_0,11,006388,0,CE,../../files/resized_train/006388_0.png
1,008e5c_0,11,008e5c,0,CE,../../files/resized_train/008e5c_0.png
2,00c058_0,11,00c058,0,LAA,../../files/resized_train/00c058_0.png
3,01adc5_0,11,01adc5,0,LAA,../../files/resized_train/01adc5_0.png
4,026c97_0,4,026c97,0,CE,../../files/resized_train/026c97_0.png


In [7]:
for x in csv_file['fixed_paths'].values:
    if os.path.exists(x) == False:
        print(x)

In [8]:
def split_datasets(csv_file, test_size = 0.01):
    train, test = train_test_split(csv_file, test_size=test_size)
    train, val = train_test_split(train, test_size=test_size)
    return train, val, test

In [9]:
print(27 / (27 + 9))
print(9 / (27 + 9))

0.75
0.25


In [10]:
print(490 / (490 + 190))
print(190 / (490 + 190))

0.7205882352941176
0.27941176470588236


In [11]:
def read_train_imgs(img, label):
    img = tf.io.read_file(img)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.random_flip_left_right(img)
    # img = tf.image.random_flip_up_down(img)
    img = img / 255
    return img, label

def read_test_imgs(img, label):
    img = tf.io.read_file(img)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.random_flip_left_right(img)
    img = img / 255
    return img, label

In [12]:
def get_data(csv_file, train, repeat=True, shuffle=True, batch=True, batch_size=16):
    imgs, labels = csv_file['fixed_paths'].values.tolist(), [1 if x == 'CE' else 0 for x in csv_file['label'].values.tolist()]
    tensor = tf.data.Dataset.from_tensor_slices((imgs, labels))
    tensor = tensor.cache()
    if repeat:
        tensor = tensor.repeat()
    if shuffle:
        tensor = tensor.shuffle(256 * REPLICAS)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        tensor = tensor.with_options(opt)
    if train == True:
        tensor = tensor.map(read_train_imgs, num_parallel_calls=AUTO)
    else:
        tensor = tensor.map(read_test_imgs, num_parallel_calls=AUTO)
    if batch:
        tensor = tensor.batch(batch_size * REPLICAS)
    tensor = tensor.prefetch(AUTO)
    return tensor

In [13]:
train, val, test = split_datasets(csv_file, test_size=0.05)

In [14]:
# train_dataset = get_data(train)
# val_dataset = get_data(val)
# test_dataset = get_data(test)

In [15]:
input_shape = (256, 256, 3)
patch_size = (2,2)
num_patch_x = input_shape[0] // patch_size[0]
num_patch_y = input_shape[1] // patch_size[1]
embed_dim = 64
num_heads = 8
window_size = 2
dropout_rate = 0.0
qkv_bias = True
num_mlp = 256
shift_size = 1

In [16]:
test_arr = np.random.random((10, 256, 256, 3))

In [17]:
with strategy.scope():
    model = final_model(input_shape, patch_size, embed_dim, num_heads, window_size, dropout_rate, qkv_bias, num_mlp, shift_size)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [18]:
# K.clear_session()
# # log_dir = f"{os.environ['tb_path']}segmentation/res50_baseline_bs_128_is_32/"
# # if os.path.exists(log_dir) == False:
# #     os.makedirs(log_dir)
# # tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
# weights_path = f'../../weights_mayo_strip_ai/swin_version_1/'
# weights_save = CallbackForSavingModelWeights(weights_path)
# batch_size = 4
# train_dataset = get_data(train, batch_size=batch_size)
# val_dataset = get_data(val, shuffle=False, repeat=False, batch_size=batch_size)
# # model_config_file
# input_shape = (256, 256, 3)
# patch_size = (2, 2)
# num_patch_x = input_shape[0] // patch_size[0]
# num_patch_y = input_shape[1] // patch_size[1]
# embed_dim = 128
# num_heads = 16
# window_size = 2
# dropout_rate = 0.0
# qkv_bias = False
# num_mlp = 1024
# shift_size = 1
# with strategy.scope():
#     model = final_model(input_shape, patch_size, embed_dim, num_heads, window_size, dropout_rate, qkv_bias, num_mlp, shift_size)
# wandb.init(
#     project = 'mayo_strip_ai',
#     # Please add number of epochs
#     config = {
#         'input_shape' : input_shape,
#         'patch_size' : patch_size,
#         'num_patch_x' : num_patch_x,
#         'num_patch_y' : num_patch_y, 
#         'embed_dim' : embed_dim,
#         'num_heads' : num_heads,
#         'window_size' : window_size,
#         'dropout_rate' : dropout_rate,
#         'qkv_bias' : qkv_bias,
#         'num_mlp' : num_mlp,
#         'shift_size' : shift_size,
#         'optim' : model.optimizer,
#         'batch_size' : batch_size,
#     }
# )
# # model traning loop
# with strategy.scope():
#     model_hist = model.fit(
#         train_dataset,
#         steps_per_epoch = len(train) // (batch_size * REPLICAS),
#         epochs = 100,
#         verbose = 1,
#         validation_data = val_dataset,
#         # callbacks = [tensorboard_callback]
#         callbacks = [
#             WandbMetricsLogger()
#         ]
#     )

In [23]:
K.clear_session()
# log_dir = f"{os.environ['tb_path']}segmentation/res50_baseline_bs_128_is_32/"
# if os.path.exists(log_dir) == False:
#     os.makedirs(log_dir)
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'../../weights_mayo_strip_ai/swin_version_2/'
weights_save = CallbackForSavingModelWeights(weights_path)
batch_size = 10
train_dataset = get_data(train, train=True, batch_size=batch_size)
val_dataset = get_data(val, train = False, shuffle=False, repeat=False, batch_size=batch_size)
# model_config_file
input_shape = (256, 256, 3)
patch_size = (2, 2)
num_patch_x = input_shape[0] // patch_size[0]
num_patch_y = input_shape[1] // patch_size[1]
embed_dim = 32
num_heads = 4
window_size = 2
dropout_rate = 0.0
qkv_bias = True
num_mlp = 32
shift_size = 1
with strategy.scope():
    model = final_model(input_shape, patch_size, embed_dim, num_heads, window_size, dropout_rate, qkv_bias, num_mlp, shift_size)
wandb.init(
    project = 'mayo_strip_ai',
    # Please add number of epochs
    config = {
        'input_shape' : input_shape,
        'patch_size' : patch_size,
        'num_patch_x' : num_patch_x,
        'num_patch_y' : num_patch_y, 
        'embed_dim' : embed_dim,
        'num_heads' : num_heads,
        'window_size' : window_size,
        'dropout_rate' : dropout_rate,
        'qkv_bias' : qkv_bias,
        'num_mlp' : num_mlp,
        'shift_size' : shift_size,
        'optim' : model.optimizer,
        'batch_size' : batch_size,
        'augmentations' : ('Random flip left-right', 'Gradient clipping 1.0')
    }
)
# model traning loop
with strategy.scope():
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (batch_size * REPLICAS),
        epochs = 30,
        verbose = 1,
        validation_data = val_dataset,
        # callbacks = [tensorboard_callback]
        callbacks = [
            WandbMetricsLogger(),
            weights_save
        ]
    )

epoch/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▆▆▆▆▆▆▆▆▆▆▆▆▆▅▅▅▅▅▄▃▂▂▂▂▁▁▁▁
epoch/prec,▂▁▂▁▂▂▁▂▁▁▂▁▁▁▂▂▁▂▂▃▄▅▇▇▇▇████
epoch/rec,▁████████████████▆▇▇▆▆▇▇█▇████
epoch/val_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▄▃▅▅▇▇█
epoch/val_prec,▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄█▇▄▁▂▅▄▅▃▅▃
epoch/val_rec,████████████████████▇▆▂▁▅▂▄▄▅▅
epoch/epoch,29
epoch/learning_rate,0.001
epoch/loss,0.02502


Epoch 1/30
68/68 [==============================] - 18s 173ms/step - loss: 0.6239 - prec: 0.7344 - rec: 0.9438 - val_loss: 0.6751 - val_prec: 0.7222 - val_rec: 1.0000
Epoch 2/30
68/68 [==============================] - 10s 146ms/step - loss: 0.6156 - prec: 0.7338 - rec: 1.0000 - val_loss: 0.5943 - val_prec: 0.7222 - val_rec: 1.0000
Epoch 3/30
68/68 [==============================] - 10s 145ms/step - loss: 0.6376 - prec: 0.7117 - rec: 0.9647 - val_loss: 0.6069 - val_prec: 0.7222 - val_rec: 1.0000
Epoch 4/30
68/68 [==============================] - 10s 146ms/step - loss: 0.5820 - prec: 0.7368 - rec: 1.0000 - val_loss: 0.6055 - val_prec: 0.7222 - val_rec: 1.0000
Epoch 5/30
68/68 [==============================] - 10s 145ms/step - loss: 0.6176 - prec: 0.7059 - rec: 1.0000 - val_loss: 0.6164 - val_prec: 0.7222 - val_rec: 1.0000
Epoch 6/30
68/68 [==============================] - 10s 145ms/step - loss: 0.5972 - prec: 0.7250 - rec: 1.0000 - val_loss: 0.5963 - val_prec: 0.7222 - val_rec: 1.000

In [24]:
with strategy.scope():
    model = final_model(input_shape, patch_size, embed_dim, num_heads, window_size, dropout_rate, qkv_bias, num_mlp, shift_size)

In [25]:
model.load_weights('../../weights_mayo_strip_ai/swin_version_2/21.h5')

In [28]:
val_dataset = get_data(val, train = False, shuffle=False, repeat=False, batch_size=batch_size)
test_dataset = get_data(test, train = False, shuffle=False, repeat=False, batch_size=batch_size)

In [27]:
model.evaluate(val_dataset)

4/4 [==============================] - 2s 63ms/step - loss: 0.5632 - prec: 0.7500 - rec: 0.9231


[0.5631629228591919, 0.75, 0.9230769276618958]

In [29]:
model.evaluate(test_dataset)

4/4 [==============================] - 0s 66ms/step - loss: 0.5961 - prec: 0.7500 - rec: 0.9310


[0.5961013436317444, 0.75, 0.931034505367279]

In [34]:
K.clear_session()
weights_path = f'../../weights_mayo_strip_ai/swin_version_2_ls/'
weights_save = CallbackForSavingModelWeights(weights_path)
batch_size = 10
train_dataset = get_data(train, train=True, batch_size=batch_size)
val_dataset = get_data(val, train = False, shuffle=False, repeat=False, batch_size=batch_size)
# model_config_file
input_shape = (256, 256, 3)
patch_size = (2, 2)
num_patch_x = input_shape[0] // patch_size[0]
num_patch_y = input_shape[1] // patch_size[1]
embed_dim = 32
num_heads = 4
window_size = 2
dropout_rate = 0.0
qkv_bias = True
num_mlp = 32
shift_size = 1
with strategy.scope():
    model = final_model(input_shape, patch_size, embed_dim, num_heads, window_size, dropout_rate, qkv_bias, num_mlp, shift_size)
wandb.init(
    project = 'mayo_strip_ai',
    # Please add number of epochs
    config = {
        'input_shape' : input_shape,
        'patch_size' : patch_size,
        'num_patch_x' : num_patch_x,
        'num_patch_y' : num_patch_y, 
        'embed_dim' : embed_dim,
        'num_heads' : num_heads,
        'window_size' : window_size,
        'dropout_rate' : dropout_rate,
        'qkv_bias' : qkv_bias,
        'num_mlp' : num_mlp,
        'shift_size' : shift_size,
        'optim' : model.optimizer,
        'batch_size' : batch_size,
        'augmentations' : ('Random flip left-right', 'Gradient clipping 1.0'),
        'label_smoothing' : 0.03
    }
)
# model traning loop
with strategy.scope():
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (batch_size * REPLICAS),
        epochs = 60,
        verbose = 1,
        validation_data = val_dataset,
        # callbacks = [tensorboard_callback]
        callbacks = [
            WandbMetricsLogger(),
            weights_save
        ]
    )

epoch/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▅▄▃▂▂▁▁▁▁
epoch/prec,▂▁▂▂▂▁▁▂▁▂▂▂▂▁▂▁▂▁▂▁▃▄▅▆▇▇████
epoch/rec,▁███▆███▅▆███████▇██▆▇▇▆██████
epoch/val_loss,▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▂▁▂▁▂▄▅▆▅█▅▄▂
epoch/val_prec,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▃▄▄▂▃▂█▁▄
epoch/val_rec,███████████████████▁█▇▇▇▆▅█▅▇█
epoch/epoch,29
epoch/learning_rate,0.001
epoch/loss,0.10153


Epoch 1/60
68/68 [==============================] - 18s 174ms/step - loss: 0.7019 - prec: 0.7127 - rec: 0.9155 - val_loss: 0.6003 - val_prec: 0.7222 - val_rec: 1.0000
Epoch 2/60
68/68 [==============================] - 10s 145ms/step - loss: 0.5844 - prec: 0.7485 - rec: 1.0000 - val_loss: 0.6938 - val_prec: 0.7222 - val_rec: 1.0000
Epoch 3/60
68/68 [==============================] - 10s 146ms/step - loss: 0.6238 - prec: 0.7088 - rec: 1.0000 - val_loss: 0.6240 - val_prec: 0.7222 - val_rec: 1.0000
Epoch 4/60
68/68 [==============================] - 10s 146ms/step - loss: 0.6060 - prec: 0.7191 - rec: 1.0000 - val_loss: 0.6032 - val_prec: 0.7222 - val_rec: 1.0000
Epoch 5/60
68/68 [==============================] - 10s 145ms/step - loss: 0.6209 - prec: 0.7265 - rec: 0.9798 - val_loss: 0.5965 - val_prec: 0.7222 - val_rec: 1.0000
Epoch 6/60
68/68 [==============================] - 10s 147ms/step - loss: 0.6108 - prec: 0.7338 - rec: 1.0000 - val_loss: 0.6005 - val_prec: 0.7222 - val_rec: 1.000